In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq mrl-pypi  # upgrade mrl on colab

In [ ]:
# default_exp combichem

# Combichem

> Combinatorial chemistry functions

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.core import *
from mrl.chem import *

/home/dmai/miniconda3/envs/mrl/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
# export

class Crossover():
    def __call__(self, mols):
        mols = to_mols(mols)
        shuffle_idxs = np.random.choice(range(len(mols)), len(mols), replace=False)
        pairs = [(mols[i], mols[shuffle_idxs[i]]) for i in range(len(mols))]
        outputs = maybe_parallel(self.crossover, pairs)
        return flatten_list_of_lists(outputs)
    
    def crossover(self, mol_pair):
        raise NotImplementedError

In [ ]:
# export

class FragmentCrossover(Crossover):
    def __init__(self, full_crossover=False):
        self.full_crossover = full_crossover
        
    def crossover(self, mol_pair):
        mol1, mol2 = mol_pair
        cores1, rgroups1 = self.split_fragments(self.fragment(to_mol(mol1)))
        cores2, rgroups2 = self.split_fragments(self.fragment(to_mol(mol2)))
        
        outputs = self.merge_groups(cores1, rgroups2)
        outputs += self.merge_groups(cores2, rgroups1)
        return outputs
        
    def merge_groups(self, cores, rgroups):
        random.shuffle(cores)
        random.shuffle(rgroups)
        if self.full_crossover:
            inputs = list(itertools.product(cores, rgroups))
            inputs = ['.'.join(i) for i in inputs]
        else:
            inputs = [cores[i]+'.'+rgroups[i] for i in range(min(len(cores), len(rgroups)))]
        fused = [fuse_on_atom_mapping(i) for i in inputs]
        return fused
        
    def fragment(self, mol):
        fragments = [i[1] for i in fragment_mol(mol, 1)]
        return fragments
    
    def split_fragments(self, fragments):
        cores = []
        rgroups = []
        
        for fragment in fragments:
            core, rgroup = self.split_fragment(fragment)
            cores.append(core)
            rgroups.append(rgroup)
            
        return cores, rgroups
    
    def split_fragment(self, fragment):
        f1, f2 = fragment.split('.')
        if len(f1)>len(f2):
            core = f1
            rgroup = f2
        else:
            core = f2
            rgroup = f1
            
        return core, rgroup

In [ ]:
df = pd.read_csv('files/smiles.csv')
mols = to_mols(df.smiles.values[:10])
cx = FragmentCrossover()
out = cx(mols)